In [1]:
import os
import sys
import collections
sys.path.append('..')
from utils.Generate import Generate
from utils.Evaluation_indicators import Evaluation_indicators
from configs.Generate_config import *
from peft import PeftModel
import transformers
import torch
import json
import numpy as np
from accelerate.utils import release_memory
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

2024-08-11 17:36:03.382466: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 17:36:03.417881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 17:36:03.417901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 17:36:03.418922: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 17:36:03.424993: I tensorflow/core/platform/cpu_feature_guar

In [10]:
train_path = './Train/Output/2024-05-30/14-27'
different_parameter_string = [train_path + '/' +i for i in os.listdir(train_path)]

save_string = ['./Generate/Output/Lora/' + train_path[-16:] + '/' +i for i in os.listdir(train_path)]

print('模型路径')
for i in different_parameter_string:
    print(i)
print('-------------------')
print('保存路径')
for i in save_string:
    print(i)
print('-------------------')

模型路径
./Train/Output/2024-05-30/14-27/checkpoint-100
./Train/Output/2024-05-30/14-27/checkpoint-120
./Train/Output/2024-05-30/14-27/checkpoint-140
./Train/Output/2024-05-30/14-27/checkpoint-40
./Train/Output/2024-05-30/14-27/checkpoint-60
./Train/Output/2024-05-30/14-27/checkpoint-80
-------------------
保存路径
./Generate/Output/Lora/2024-05-30/14-27/checkpoint-100
./Generate/Output/Lora/2024-05-30/14-27/checkpoint-120
./Generate/Output/Lora/2024-05-30/14-27/checkpoint-140
./Generate/Output/Lora/2024-05-30/14-27/checkpoint-40
./Generate/Output/Lora/2024-05-30/14-27/checkpoint-60
./Generate/Output/Lora/2024-05-30/14-27/checkpoint-80
-------------------


### 推理

In [3]:
test = Generate(**CONFIG)
# 读取可用模型序列
base_model = 'meta-llama/Llama-2-7b-hf'
#base_model = 'chat-gpt'
use_tokenizer = test.load_tokenizer(base_model)

Loading Tokenizer : "meta-llama/Llama-2-7b-hf"
Tokenizer: meta-llama/Llama-2-7b-hf Loaded Successfully


Beginning token : <s> , Beginning token id : 1 

Ending token : </s> , Ending token id : 2 

Padding token : None , Padding token id : None 



In [4]:
PROMPTER_first_question = test.load_template(prompt_template_path = "alpaca")
#PROMPTER_other_questions = test.load_template(prompt_template_path = "alpaca")
predict_data = test.load_data("/root/autodl-tmp/new_version/test_set_fixed.json")
#predict_data = test.load_data("../Train/Dataset/Fast_test/Test_Set.json")
pre_data = test.Preprocess()
b_data,o_data = test.map_dataset(test.data_preprocess,add_eos_token=True)

Loading template : alpaca
Template Loaded Successfully
Loading data: "/root/autodl-tmp/new_version/test_set_fixed.json"
Predict data loaded successfully
Processing Data.
Data Preprocess Done
Start mapping data.
Add eos token in the end of inputs_id.
Data Mapped Successfully


In [5]:
loaded_model = test.load_model()
generation_config = test.Parameter_setting()

has tokenizer
Loading Model : "meta-llama/Llama-2-7b-hf"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model: meta-llama/Llama-2-7b-hf --- Loaded Successfully
Without lora adaption.
Parameter setting Done


/root/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/transformers/gene

In [ ]:
for i in different_parameter_string:
    print(i)
    index = i.split('/')[-1]
    model_lora = PeftModel.from_pretrained(
        loaded_model,
        i,
        torch_dtype=torch.float32,        
    )
    
    for j in ['o_data']:
        p_d = test.predcit_circulate(eval(j),is_shut_down=False,shut_down_index=1)
        save_file = p_d.remove_columns(column_names=['input_ids',])
        save_path = save_string[different_parameter_string.index(i)]
        test.save_results(save_file,output_dir=save_path)

### 评估

In [7]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [29]:
baseline_string = ['Generate/Output/chat-gpt/06-16']
Evluator = Evaluation_indicators()
evaluete_string = save_string
print('评估路径')
evaluete_string

评估路径


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['./Generate/Output/Lora/2024-05-30/14-27/checkpoint-40',
 './Generate/Output/Lora/2024-05-30/14-27/checkpoint-60',
 './Generate/Output/Lora/2024-05-30/14-27/checkpoint-80',
 './Generate/Output/Lora/2024-05-30/14-27/checkpoint-100',
 './Generate/Output/Lora/2024-05-30/14-27/checkpoint-120',
 './Generate/Output/Lora/2024-05-30/14-27/checkpoint-140']

In [34]:
for i in evaluete_string:
    for j in ['/Only/']:
        path = i + j
        json_path = path + os.listdir(path)[0]
        print(json_path)  # 确认路径正确
        evluate_data = Evluator.load_data(json_path)

        # 打印加载数据的一部分来检查结构
        print(evluate_data[0])  # 打印第一条数据看看结构

        # 假设 output_types 定义了每种输出类型的列名
        output_types = {
            'entity1': 'Entity1:Object',
            'entity2': 'Entity2:Spectrum',
            'question_type': 'Question_type'
        }

        choose_methods=None
        # 调用 run_evaluate 方法
        result = Evluator.run_evaluate(evluate_data, choose_methods, output_types)
        print(result)  # 打印结果以便检查

        # 保存结果为json文件
        Evluator.save_json(result, path + 'result.json')

./Generate/Output/Lora/2024-05-30/14-27/checkpoint-40/Only/17_50_00_fixed.json
Loading data: "./Generate/Output/Lora/2024-05-30/14-27/checkpoint-40/Only/17_50_00_fixed.json"
Predict data loaded successfully
{'Output': {'Entity1:Object': 'octopus', 'Entity2:Spectrum': 'nir, near-infrared', 'Question_type': 'Random questions'}, 'model_response': "{'Entity1:Object': 'fish', 'Entity2:Spectrum': 'not found', 'Question_type': 'Evaluated metrics'}</s>", 'Instruction': "Instruction: Please extract the research object entities and spectral types used in the input questions, and classify the types of questions.The types of questions are: 1. Spectral Detection Method 2. Feature Processing Method 3. Machine learning Method 4. Evaluated metrics 5. Random questions.\n For the input questions which belongs to the type of Spectral Detection Method, the spectral types extracted should be 'not found'.", 'Input': 'Question:How do the specific characteristics of fishing waters contribute to the homogeneit

### 读取评估结果

In [33]:
from datasets import load_dataset
kkk = load_dataset('json', data_files='Generate/Output/Lora/2024-05-30/14-27/checkpoint-140/Only/result.json')
kkk['train'].to_pandas()

Generating train split: 0 examples [00:00, ? examples/s]

,entity1_bleu,entity1_rouge1,entity1_rouge2,entity1_rougeL,entity1_meteor,entity1_bert_score_precision,entity1_bert_score_recall,entity1_bert_score_f1,entity2_accuracy,entity2_bleu,...,entity2_bert_score_f1,question_type_accuracy,question_type_bleu,question_type_rouge1,question_type_rouge2,question_type_rougeL,question_type_meteor,question_type_bert_score_precision,question_type_bert_score_recall,question_type_bert_score_f1
0,0.087005,0.527546,0.326446,0.525624,0.439851,0.910977,0.901378,0.905218,0.882474,0.422933,...,0.973484,0.97732,0.0,0.978007,0.97732,0.978007,0.942607,0.996799,0.996404,0.996592
